# Real Time Location System

### Predicting Location with K- Nearest Neighbor

#### Team: 
- Bin Yu
- Nuoya Rezsonya
- Yejur SIngh Kunwar
- Iram Bakhtiar

# DELIVERABLES 
### ****MAIN QUESTIONS TO ANSWER FROM THIS CASE STUDY

- Conduct a more thorough data analysis into these two MAC addresses including determining locations by using data corresponding to both MAC addresses.  

- Which of these two MAC addresses should be used and which should not be used for RTLS? 
- Which MAC address yields the best prediction of location?  
- Does using data for both MAC addresses simultaneously yield more, or less, accurate prediction of location? (Note: this portion is derived from Exercise Q.9 in Nolan and Lang.)

- While k-nearest neighbors has proven to be a good approach to determining location, alternate approaches have been proposed.  One simple alternative approach is to use weights on the received signal strength, where the weight is inversely proportional to the “distance” from the test observation.  This allows for the “nearest” points to have a greater contribution to the k-nearest neighbor location calculation than the points that are “further” away.  

- Implement this alternative prediction method.  
- For what range of values of weights are you able to obtain better prediction values than for the unweighted k-nearest neighbor approach? Use calcError() to compare this approach to the simple average.


# STRUCTURE OF REPORT

- Introduction: Explain the case study 
- Methodology : Explain the approach used to complete the case study 
- Results     : Explain the output achieved (Explanations of output should be included as close to the output or figures as possible.)
- References

### Introduction ( we will add this part later)

Introduction part

### Methodology (we will add this part later)

Talking about what we used in this notebook and reference the resource 

### Data Import

This part will be words to describe the data source as in where it comes from, dimension of the data, column explaination(which is what)

# CODE AND COMMENTS

In [3]:
# Case Study from: Data Science in R: Nolan,Temple,Lang (Ch 1)

# Source code: http://rdatasciencecases.org/code.html
# Dataset: http://rdatasciencecases.org/Data.html

# Installing and importing necessary libraries

# install.packages("fields")  (RUN THIS IF YOU DO NOT HAVE PACKAGE)
# library(fields)
library(rstudioapi)
library(lattice)




In [4]:
# Working directory setup
options(digits = 2)
current_path <- getActiveDocumentContext()$path
setwd(dirname(current_path ))

ERROR: Error: RStudio not running


### DATA EXPLORATION

In [5]:
txt = readLines("Data/offline.final.trace.txt")
length(txt)    # Loaded 151392 lines/strings
head(txt,4)

[1] 151392

[1] "# timestamp=2006-02-11 08:31:58"                                                                                                                                                                                                                                                                                                                                                                                                                                 
[2] "# usec=250"                                                                                                                                                                                                                                                                                                                                                                                                                                                      
[3] "# minReadings=110"                                                                                                                                                                                                                                                                                                                                                                                                                                               
[4] "t=1139643118358;id=00:02:2D:21:0F:33;pos=0.0,0.0,0.0;degree=0.0;00:14:bf:b1:97:8a=-38,2437000000,3;00:14:bf:b1:97:90=-56,2427000000,3;00:0f:a3:39:e1:c0=-53,2462000000,3;00:14:bf:b1:97:8d=-65,2442000000,3;00:14:bf:b1:97:81=-65,2422000000,3;00:14:bf:3b:c7:c6=-66,2432000000,3;00:0f:a3:39:dd:cd=-75,2412000000,3;00:0f:a3:39:e0:4b=-78,2462000000,3;00:0f:a3:39:e2:10=-87,2437000000,3;02:64:fb:68:52:e6=-88,2447000000,1;02:00:42:55:31:00=-84,2457000000,1"

In [6]:
# Prelimniary data exploration for dimensions
print(paste("The total number of lines in offline data =",length(txt)))
print(paste("The number of comments in data =", sum(substr(txt, 1, 1) == "#"))) # there are comments in the data, figure out amount
print(paste("The actual number of data, non-comment lines =", length(txt) - sum(substr(txt, 1, 1) == "#"))) # the actual data lines


[1] "The total number of lines in offline data = 151392"
[1] "The number of comments in data = 5312"
[1] "The actual number of data, non-comment lines = 146080"


In [7]:
# Parsing samples and splitting on identified charcters line [;=,] 
tokens = strsplit(txt[4], "[;=,]")[[1]]
tokens[1:10]
tokens[c(2, 4, 6:8, 10)]

[1] "t"                 "1139643118358"     "id"               
 [4] "00:02:2D:21:0F:33" "pos"               "0.0"              
 [7] "0.0"               "0.0"               "degree"           
[10] "0.0"

[1] "1139643118358"     "00:02:2D:21:0F:33" "0.0"              
[4] "0.0"               "0.0"               "0.0"

In [9]:
# Displayinh variables MACID; Signal Strength, Freq., Mode
tokens[ - ( 1:10 ) ]
# Formatting as matrix to create IDs,
tmp = matrix(tokens[ - (1:10) ], ncol = 4, byrow = TRUE)
mat = cbind(matrix(tokens[c(2, 4, 6:8, 10)], 
                           nrow = nrow(tmp),
                           ncol = 6, byrow = TRUE), tmp)
        
# Final matrix
dim(mat)

[1] "00:14:bf:b1:97:8a" "-38"               "2437000000"       
 [4] "3"                 "00:14:bf:b1:97:90" "-56"              
 [7] "2427000000"        "3"                 "00:0f:a3:39:e1:c0"
[10] "-53"               "2462000000"        "3"                
[13] "00:14:bf:b1:97:8d" "-65"               "2442000000"       
[16] "3"                 "00:14:bf:b1:97:81" "-65"              
[19] "2422000000"        "3"                 "00:14:bf:3b:c7:c6"
[22] "-66"               "2432000000"        "3"                
[25] "00:0f:a3:39:dd:cd" "-75"               "2412000000"       
[28] "3"                 "00:0f:a3:39:e0:4b" "-78"              
[31] "2462000000"        "3"                 "00:0f:a3:39:e2:10"
[34] "-87"               "2437000000"        "3"                
[37] "02:64:fb:68:52:e6" "-88"               "2447000000"       
[40] "1"                 "02:00:42:55:31:00" "-84"              
[43] "2457000000"        "1"

[1] 11 10